In [1]:
import numpy as np
import h5py
import scipy
from scipy.signal import wiener, medfilt
import pandas as pd
import matplotlib.pyplot as plt
from skimage.util.montage import montage2d
from skimage.color import label2rgb
import os
import pickle
%matplotlib inline


In [3]:
loaded_model = pickle.load(open('finalized_model.sav', 'rb'))

In [4]:
ct_data, pet_data, label_data = [], [], []
num_data = 1
with h5py.File(os.path.join('..','input','lab_petct_vox_5.00mm.h5'),'r') as p_data:
    for ct_img, pet_img, label in zip(p_data['ct_data'].values(),
                                      p_data['pet_data'].values(),
                                      p_data['label_data'].values()):
        #print(np.shape(ct_img))
        #print(len(ct_img))
        ct_data += [np.array(ct_img)]
        pet_data += [np.array(pet_img)]
        label_data += [np.array(label)]
        num_data = num_data - 1
        if num_data == 0:
            break

In [5]:
ct_cubes, pet_cubes,lab_cubes = [], [], []
tile_size=16
stride=16
for ct_img, pet_img, label in zip(ct_data, pet_data, label_data):
    for i in range(0,  ct_img.shape[0], tile_size):
        for j in range(0,  ct_img.shape[1], tile_size):
            for z in range(0,  ct_img.shape[2], tile_size):
                for i_k in range(0, tile_size, stride):
                    for j_k in range(0, tile_size, stride):
                        for z_k in range(0,tile_size,stride):
                            ct_cubes += [ct_img[i+i_k:(i+i_k+tile_size),j+j_k:(j+j_k+tile_size),z+z_k:(z+z_k+tile_size)]]
                            pet_cubes += [pet_img[i+i_k:(i+i_k+tile_size),j+j_k:(j+j_k+tile_size),z+z_k:(z+z_k+tile_size)]]
                            lab_cubes += [label[i+i_k:(i+i_k+tile_size),j+j_k:(j+j_k+tile_size),z+z_k:(z+z_k+tile_size)]] 


In [6]:
# The following functions take a cube and an axis and return a projection of the tile along the axis
from sklearn.preprocessing import normalize

def reduceCTCube(cube,axis):
    return np.sum(cube,axis)
    
def reducePETCube(cube,axis):
    #if(np.sqrt(np.max(cube,axis)==nan)):
        #print(mk,axis)
    cube_array=np.sqrt(np.max(cube,axis))
    where_are_NaNs = np.isnan(cube_array)
    #print([where_are_NaNs])
    cube_array[where_are_NaNs] = 0.001386386961242742
    #return np.sqrt(np.max(cube,axis))
    #if np.any(np.isnan(cube_array)):
        #print('sad')
    return cube_array
    
def reduceLabelCube(cube):
    # write here, returns a single value, not a list/array    
    return(np.mean(cube))

In [7]:
data_tiles, data_labels = [], []

n_ct_cubes = [c_tile for c_tile in ct_cubes if c_tile.shape == (tile_size,tile_size,tile_size)]
n_pet_cubes = [c_tile for c_tile in pet_cubes if c_tile.shape == (tile_size,tile_size,tile_size)]
n_lab_cubes = [c_tile for c_tile in lab_cubes if c_tile.shape == (tile_size,tile_size,tile_size)]

#ct_cube_final, pet_cube_final, lab_cube_final = zip(*[(c_cube, p_cube, l_cube) 
      #for c_cube, p_cube, l_cube in zip(n_ct_cubes, n_pet_cubes, n_lab_cubes)
     #if np.mean(c_cube) > -1001 and np.mean(p_cube) > .0002])
#print('%d tiles removed' % (len(n_ct_cubes)-len(ct_cube_final)))

#print(len(pet_cube_final))

#for ct_cube, pet_cube, lab_cube in zip(ct_cube_final, pet_cube_final,lab_cube_final):
#mk=0
for ct_cube, pet_cube, lab_cube in zip(n_ct_cubes, n_pet_cubes,n_lab_cubes):
    data_tile = []
    for axis in range(3):
        a=reducePETCube(pet_cube, axis).flatten()
        data_tile = np.concatenate([data_tile, a])
                
    #mk=mk+1
    #print()
    data_tiles += [data_tile]
    data_labels += [reduceLabelCube(lab_cube)]
print(len(n_ct_cubes))    

360


C:\Users\Himanshu\Miniconda3\lib\site-packages\ipykernel\__main__.py:10: RuntimeWarning: invalid value encountered in sqrt


In [8]:
#result=loaded_model.score
#plt.imshow(ct_cubes)
#np.shape(ct_cubes)